# Google API

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Prerequisite:
Create a google oauth2 credential, download it the this folder as credentials.json. Follow this [guide]https://developers.google.com/workspace/guides/create-credentials

This code will convert it into token.json, which will be used to call google apis.

In [1]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow


SCOPES = ['https://www.googleapis.com/auth/documents']

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())


Build a new document

In [2]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

document_id = ""
try:
    service = build('docs', 'v1', credentials=creds)
    title = 'My Document 3'
    body = {
        'title': title,
    }
    response = service.documents().create(body=body).execute()
    print('Created document with title: {0}'.format(response.get('title')))
    document_id = response['documentId']
    print(document_id)
except HttpError as err:
    print(err)

Created document with title: My Document 3
1dgnYPeVIAF613WMx2v8rPOZPTgUPX41sDrErLDJEmtM


Update the new document

In [3]:
try:
    requests = [
        {
            "insertText": {
                "text": "Sample1\n",
                "location": {
                    "index": 1
                }
            }
        },
        {
            "insertText": {
                "text": "sample2\n",
                "location": {
                    "index": 9
                }
            }
        }
    ]
    result = service.documents().batchUpdate(
        documentId=document_id, body={'requests': requests}).execute()
except HttpError as err:
    print(err)

# Summarize google doc using langchain

Load the google doc using langchain's google api.

Get the llm model.

Create a custom chain and run the query.

In [10]:
from langchain.document_loaders import GoogleDriveLoader
import os

loader = GoogleDriveLoader(document_ids=["1PqfBr0zti7O2_n0Sr6cXDeFSTAqbAH6AteARRsgZWhw"], credentials_path="./credentials.json", token_path="./token.json")
docs = loader.load()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=274229182110-41qo6pk107o34b005ilamjk9undt963v.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52138%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=LnnTOvMrT5Fa8xauht3hqCp1i7EIOe&access_type=offline


In [11]:
docs[0].page_content[:100]

'\ufeffHi, everyone. Welcome toThis is paragraph 1. GrayThis is paragraph 2. MattThis is paragraph 3.er, t'

In [12]:
from langchain.chat_models import ChatOpenAI

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY') or 'OPENAI_API_KEY'

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [13]:
from langchain import PromptTemplate

template = """
Summarize this text:

{output_text}

Summarize: """

prompt = PromptTemplate(input_variables=["output_text"], template=template)

In [14]:
from langchain.chains import LLMChain

llm_chain = LLMChain(llm=llm, prompt=prompt, output_key="summary")

In [15]:
llm_chain.run(docs[0].page_content)